## CHAPTER 10
# Data Aggregation and Group Operations

In [45]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

## 10.1 How to Think About Group Operations

In [46]:
df = pd.DataFrame(
    {
        "key1": ["a", "a", None, "b", "b", "a", None],
        "key2": pd.Series([1, 2, 1, 2, 1, None, 1], dtype="Int64"),
        "data1": np.random.standard_normal(7),
        "data2": np.random.standard_normal(7),
    }
)
df

,key1,key2,data1,data2
0,a,1,-0.351227,1.373748
1,a,2,-0.259201,0.009464
2,None,1,-0.240990,1.630072
3,b,2,-0.240628,-0.291756
4,b,1,0.107098,-0.402465
5,a,<NA>,-0.173548,0.287442
6,None,1,-0.663357,0.427762


In [47]:
grouped = df["data1"].groupby(df["key1"])
grouped

In [48]:
grouped.mean()

key1
a   -0.261325
b   -0.066765
Name: data1, dtype: float64

In [49]:
means = df["data1"].groupby([df["key1"], df["key2"]]).mean()
means

key1  key2
a     1      -0.351227
      2      -0.259201
b     1       0.107098
      2      -0.240628
Name: data1, dtype: float64

In [50]:
means.unstack()

key2,1,2
key1,,
a,-0.351227,-0.259201
b,0.107098,-0.240628


In [51]:
state = np.array(["OH", "CA", "CA", "OH", "OH", "CA", "OH"])
years = np.array([2005, 2005, 2006, 2005, 2006, 2005, 2006])
df["data1"].groupby([state, years]).mean()

CA  2005   -0.216375
    2006   -0.240990
OH  2005   -0.295928
    2006   -0.278130
Name: data1, dtype: float64

In [52]:
df.groupby("key1").mean()

,key2,data1,data2
key1,,,
a,1.5,-0.261325,0.556885
b,1.5,-0.066765,-0.347110


In [53]:
df.groupby("key2").mean(numeric_only=True)

,data1,data2
key2,,
1,-0.287119,0.757280
2,-0.249915,-0.141146


In [54]:
df.groupby(["key1", "key2"]).mean()

data1     data2
key1 key2                    
a    1    -0.351227  1.373748
     2    -0.259201  0.009464
b    1     0.107098 -0.402465
     2    -0.240628 -0.291756

In [55]:
df.groupby(["key1", "key2"]).size()

key1  key2
a     1       1
      2       1
b     1       1
      2       1
dtype: int64

In [56]:
df.groupby("key1", dropna=False).size()

key1
a      3
b      2
NaN    2
dtype: int64

In [57]:
df.groupby(["key1", "key2"], dropna=False).size()

key1  key2
a     1       1
      2       1
      <NA>    1
b     1       1
      2       1
NaN   1       2
dtype: int64

In [58]:
df.groupby("key1").count()

,key2,data1,data2
key1,,,
a,2,3,3
b,2,2,2


### Iterating over Groups

In [59]:
for name , group in df.groupby("key1"):
    print(name)
    print(group)

a
  key1  key2     data1     data2
0    a     1 -0.351227  1.373748
1    a     2 -0.259201  0.009464
5    a  <NA> -0.173548  0.287442
b
  key1  key2     data1     data2
3    b     2 -0.240628 -0.291756
4    b     1  0.107098 -0.402465


In [60]:
for (k1, k2), group in df.groupby(["key1", "key2"]):
    print((k1, k2))
    print(group)

('a', np.int64(1))
  key1  key2     data1     data2
0    a     1 -0.351227  1.373748
('a', np.int64(2))
  key1  key2     data1     data2
1    a     2 -0.259201  0.009464
('b', np.int64(1))
  key1  key2     data1     data2
4    b     1  0.107098 -0.402465
('b', np.int64(2))
  key1  key2     data1     data2
3    b     2 -0.240628 -0.291756


In [61]:
pieces = {name: group for name, group in df.groupby("key1")}
pieces["b"]

,key1,key2,data1,data2
3,b,2,-0.240628,-0.291756
4,b,1,0.107098,-0.402465


In [62]:
pieces

{'a':   key1  key2     data1     data2
 0    a     1 -0.351227  1.373748
 1    a     2 -0.259201  0.009464
 5    a  <NA> -0.173548  0.287442,
 'b':   key1  key2     data1     data2
 3    b     2 -0.240628 -0.291756
 4    b     1  0.107098 -0.402465}

In [63]:
grouped = df.groupby({"key1": "key", "key2": "key",
                      "data1": "data", "data2": "data"}, axis="columns")
for group_key, group_values in grouped:
    print(group_key)
    print(group_values)

data
      data1     data2
0 -0.351227  1.373748
1 -0.259201  0.009464
2 -0.240990  1.630072
3 -0.240628 -0.291756
4  0.107098 -0.402465
5 -0.173548  0.287442
6 -0.663357  0.427762
key
   key1  key2
0     a     1
1     a     2
2  None     1
3     b     2
4     b     1
5     a  <NA>
6  None     1


C:\Users\sahor\AppData\Local\Temp\ipykernel_17860\695880334.py:1: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  grouped = df.groupby({"key1": "key", "key2": "key",


### Selecting a Column or Subset of Columns

In [64]:
df

,key1,key2,data1,data2
0,a,1,-0.351227,1.373748
1,a,2,-0.259201,0.009464
2,None,1,-0.240990,1.630072
3,b,2,-0.240628,-0.291756
4,b,1,0.107098,-0.402465
5,a,<NA>,-0.173548,0.287442
6,None,1,-0.663357,0.427762


In [65]:
df.groupby("key1")["data1"]
df.groupby("key1")[["data2"]]

In [66]:
df["data1"].groupby(df["key1"])
df[["data2"]].groupby(df["key1"])

In [67]:
df.groupby(["key1", "key2"])[["data2"]].mean()

data2
key1 key2          
a    1     1.373748
     2     0.009464
b    1    -0.402465
     2    -0.291756

In [68]:
s_grouped = df.groupby(["key1", "key2"])["data2"]
s_grouped

In [69]:
s_grouped.mean()

key1  key2
a     1       1.373748
      2       0.009464
b     1      -0.402465
      2      -0.291756
Name: data2, dtype: float64

### Grouping with Dictionaries and Series

In [70]:
people = pd.DataFrame(
    np.random.randn(5, 5),
    columns=["a", "b", "c", "d", "e"],
    index=["Joe", "Steve", "Wanda", "Jill", "Trey"],
)
people.iloc[2:3, [1, 2]] = np.nan
people

,a,b,c,d,e
Joe,0.255283,0.628116,1.838243,0.084037,0.604399
Steve,1.612316,0.180524,1.364050,0.398034,-1.139662
Wanda,0.742352,NaN,NaN,-0.101708,-0.995792
Jill,0.781781,-1.327971,-0.948921,0.646627,1.116847
Trey,-0.980251,-1.143764,1.107125,-0.225192,-1.603646


In [71]:
mapping = {"a": "red", "b": "red", "c": "blue",
            "d": "blue", "e": "red", "f" : "orange"}


In [72]:
by_column = people.groupby(mapping, axis="columns")
by_column.sum()

C:\Users\sahor\AppData\Local\Temp\ipykernel_17860\2628240624.py:1: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  by_column = people.groupby(mapping, axis="columns")


,blue,red
Joe,1.922280,1.487798
Steve,1.762084,0.653178
Wanda,-0.101708,-0.253441
Jill,-0.302294,0.570657
Trey,0.881933,-3.727662


In [73]:
map_series = pd.Series(mapping)
map_series

a       red
b       red
c      blue
d      blue
e       red
f    orange
dtype: object

In [74]:
people.groupby(map_series, axis="columns").count()

C:\Users\sahor\AppData\Local\Temp\ipykernel_17860\2727320543.py:1: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  people.groupby(map_series, axis="columns").count()


,blue,red
Joe,2,3
Steve,2,3
Wanda,1,2
Jill,2,3
Trey,2,3


### Grouping with Functions

In [75]:
people

,a,b,c,d,e
Joe,0.255283,0.628116,1.838243,0.084037,0.604399
Steve,1.612316,0.180524,1.364050,0.398034,-1.139662
Wanda,0.742352,NaN,NaN,-0.101708,-0.995792
Jill,0.781781,-1.327971,-0.948921,0.646627,1.116847
Trey,-0.980251,-1.143764,1.107125,-0.225192,-1.603646


In [76]:
people.groupby(len).sum()  # group by length of the index value

,a,b,c,d,e
3,0.255283,0.628116,1.838243,0.084037,0.604399
4,-0.198470,-2.471735,0.158204,0.421435,-0.486799
5,2.354668,0.180524,1.364050,0.296326,-2.135454


In [77]:
key_list = ["one", "one", "one", "two", "two"]
people.groupby([len, key_list]).min()

,,a,b,c,d,e
3,one,0.255283,0.628116,1.838243,0.084037,0.604399
4,two,-0.980251,-1.327971,-0.948921,-0.225192,-1.603646
5,one,0.742352,0.180524,1.364050,-0.101708,-1.139662


### Grouping by Index Levels

In [78]:
columns = pd.MultiIndex.from_arrays([["US", "US", "US", "JP", "JP"],
                                     [1, 3, 5, 1, 3]],
                                    names=["cty", "tenor"])
hier_df = pd.DataFrame(np.random.randn(4, 5), columns=columns)
hier_df

cty          US                            JP          
tenor         1         3         5         1         3
0      1.453906  0.566085  2.109876  1.203690 -0.269583
1     -0.735898  0.924176 -1.150417 -0.763172 -1.264203
2      0.002512  0.663919  0.816771  2.042342  0.207327
3     -1.605345  1.419318  1.327961  1.270208 -0.089367

In [79]:
hier_df.groupby(level="cty", axis="columns").count()

C:\Users\sahor\AppData\Local\Temp\ipykernel_17860\2795167867.py:1: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  hier_df.groupby(level="cty", axis="columns").count()


cty,JP,US
0,2,3
1,2,3
2,2,3
3,2,3


## 10.2 Data Aggregation

In [80]:
df

,key1,key2,data1,data2
0,a,1,-0.351227,1.373748
1,a,2,-0.259201,0.009464
2,None,1,-0.240990,1.630072
3,b,2,-0.240628,-0.291756
4,b,1,0.107098,-0.402465
5,a,<NA>,-0.173548,0.287442
6,None,1,-0.663357,0.427762


In [81]:
grouped = df.groupby("key1")
grouped["data1"].nsmallest(2)

key1   
a     0   -0.351227
      1   -0.259201
b     3   -0.240628
      4    0.107098
Name: data1, dtype: float64

In [82]:
def peak_to_peak(arr):
    return arr.max() - arr.min()
grouped.agg(peak_to_peak)

,key2,data1,data2
key1,,,
a,1,0.177678,1.364285
b,1,0.347726,0.110709


In [83]:
grouped.describe()

key2                                           data1            ...  \
     count mean       std  min   25%  50%   75%  max count      mean  ...   
key1                                                                  ...   
a      2.0  1.5  0.707107  1.0  1.25  1.5  1.75  2.0   3.0 -0.261325  ...   
b      2.0  1.5  0.707107  1.0  1.25  1.5  1.75  2.0   2.0 -0.066765  ...   

                         data2                                          \
           75%       max count      mean       std       min       25%   
key1                                                                     
a    -0.216375 -0.173548   3.0  0.556885  0.720949  0.009464  0.148453   
b     0.020167  0.107098   2.0 -0.347110  0.078283 -0.402465 -0.374787   

                                    
           50%       75%       max  
key1                                
a     0.287442  0.830595  1.373748  
b    -0.347110 -0.319433 -0.291756  

[2 rows x 24 columns]

### Column-Wise and Multiple Function Application

In [85]:
# tips = sns.load_dataset("tips")
tips = pd.read_csv("examples/tips.csv")
tips.head()

,total_bill,tip,smoker,day,time,size
0,16.99,1.01,No,Sun,Dinner,2
1,10.34,1.66,No,Sun,Dinner,3
2,21.01,3.50,No,Sun,Dinner,3
3,23.68,3.31,No,Sun,Dinner,2
4,24.59,3.61,No,Sun,Dinner,4


In [86]:
tips["tip_pct"] = tips["tip"] / tips["total_bill"]
tips.head()

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808


In [87]:
grouped = tips.groupby(["day", "smoker"])
grouped_pct = grouped["tip_pct"]
grouped_pct.mean()

day   smoker
Fri   No        0.151650
      Yes       0.174783
Sat   No        0.158048
      Yes       0.147906
Sun   No        0.160113
      Yes       0.187250
Thur  No        0.160298
      Yes       0.163863
Name: tip_pct, dtype: float64

In [88]:
grouped_pct.agg(["mean", "std", peak_to_peak])

mean       std  peak_to_peak
day  smoker                                  
Fri  No      0.151650  0.028123      0.067349
     Yes     0.174783  0.051293      0.159925
Sat  No      0.158048  0.039767      0.235193
     Yes     0.147906  0.061375      0.290095
Sun  No      0.160113  0.042347      0.193226
     Yes     0.187250  0.154134      0.644685
Thur No      0.160298  0.038774      0.193350
     Yes     0.163863  0.039389      0.151240

In [89]:
grouped_pct.agg([("average", "mean"), ("stdev", "std")])

average     stdev
day  smoker                    
Fri  No      0.151650  0.028123
     Yes     0.174783  0.051293
Sat  No      0.158048  0.039767
     Yes     0.147906  0.061375
Sun  No      0.160113  0.042347
     Yes     0.187250  0.154134
Thur No      0.160298  0.038774
     Yes     0.163863  0.039389

In [92]:
functions = ["count", "mean", "max"]
result = grouped[["tip_pct", "total_bill"]].agg(functions)
result

tip_pct                     total_bill                  
              count      mean       max      count       mean    max
day  smoker                                                         
Fri  No           4  0.151650  0.187735          4  18.420000  22.75
     Yes         15  0.174783  0.263480         15  16.813333  40.17
Sat  No          45  0.158048  0.291990         45  19.661778  48.33
     Yes         42  0.147906  0.325733         42  21.276667  50.81
Sun  No          57  0.160113  0.252672         57  20.506667  48.17
     Yes         19  0.187250  0.710345         19  24.120000  45.35
Thur No          45  0.160298  0.266312         45  17.113111  41.19
     Yes         17  0.163863  0.241255         17  19.190588  43.11

In [93]:
result["tip_pct"]

count      mean       max
day  smoker                           
Fri  No          4  0.151650  0.187735
     Yes        15  0.174783  0.263480
Sat  No         45  0.158048  0.291990
     Yes        42  0.147906  0.325733
Sun  No         57  0.160113  0.252672
     Yes        19  0.187250  0.710345
Thur No         45  0.160298  0.266312
     Yes        17  0.163863  0.241255

In [94]:
ftuples = [("Average", "mean"), ("Variance", np.var)]
grouped[["tip_pct", "total_bill"]].agg(ftuples)

C:\Users\sahor\AppData\Local\Temp\ipykernel_17860\4198102554.py:2: FutureWarning: The provided callable <function var at 0x00000218E3DFDC60> is currently using SeriesGroupBy.var. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "var" instead.
  grouped[["tip_pct", "total_bill"]].agg(ftuples)


tip_pct           total_bill            
              Average  Variance    Average    Variance
day  smoker                                           
Fri  No      0.151650  0.000791  18.420000   25.596333
     Yes     0.174783  0.002631  16.813333   82.562438
Sat  No      0.158048  0.001581  19.661778   79.908965
     Yes     0.147906  0.003767  21.276667  101.387535
Sun  No      0.160113  0.001793  20.506667   66.099980
     Yes     0.187250  0.023757  24.120000  109.046044
Thur No      0.160298  0.001503  17.113111   59.625081
     Yes     0.163863  0.001551  19.190588   69.808518

In [95]:
grouped.agg({"tip": np.max, "size": "sum"})

C:\Users\sahor\AppData\Local\Temp\ipykernel_17860\3453032214.py:1: FutureWarning: The provided callable <function max at 0x00000218E3DFCFE0> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  grouped.agg({"tip": np.max, "size": "sum"})


tip  size
day  smoker             
Fri  No       3.50     9
     Yes      4.73    31
Sat  No       9.00   115
     Yes     10.00   104
Sun  No       6.00   167
     Yes      6.50    49
Thur No       6.70   112
     Yes      5.00    40

In [100]:
grouped.agg({"tip_pct": ["min", "max", "mean", "std"],
            "size": "sum"})

tip_pct                               size
                  min       max      mean       std  sum
day  smoker                                             
Fri  No      0.120385  0.187735  0.151650  0.028123    9
     Yes     0.103555  0.263480  0.174783  0.051293   31
Sat  No      0.056797  0.291990  0.158048  0.039767  115
     Yes     0.035638  0.325733  0.147906  0.061375  104
Sun  No      0.059447  0.252672  0.160113  0.042347  167
     Yes     0.065660  0.710345  0.187250  0.154134   49
Thur No      0.072961  0.266312  0.160298  0.038774  112
     Yes     0.090014  0.241255  0.163863  0.039389   40

### Returning Aggregated Data Without Row Indexes